In [1]:
import cv2
import time
import numpy as np
from PIL import Image
import os
import sys

In [28]:
def convertImage(filePath, outName):
    t = time.time()
    
    try:
        #the .prototxt file speficies the architecture of the neural network
        protoFile = "pose/coco/pose_deploy_linevec.prototxt"
        #the .caffemodel file stores the weight of the trained model
        weightsFile = "pose/coco/pose_iter_440000.caffemodel"
        nPoints = 18
        POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]
        #{Nose – 0, Neck – 1, Right Shoulder – 2, Right Elbow – 3, Right Wrist – 4, Left Shoulder – 5, Left Elbow – 6...}

        imageName = str(outName) + '.jpg'
        frame = cv2.imread(filePath)
        #stores an array copy of the image
        frameCopy = np.copy(frame)
        #print(frameCopy)
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        threshold = 0.1

        #read the network into memory 
        net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)


        # input image dimensions for the network
        inWidth = 368
        inHeight = 368

        #converting the image into an input blob so it can be fed into the network 
        #converts image from openCV format to Caffe blob format 
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                  (0, 0, 0), swapRB=False, crop=False)

        #set the inpBlob as the input blob for the network 
        net.setInput(inpBlob)

        #passing the image to the model
        output = net.forward()

        #output will return a 4D matrix where the first element is the image id,
        #the second element is index of a keypoint
        #the third and fourth are the height and width of the image 

        #gather height and width from the output produced
        H = output.shape[2]
        W = output.shape[3]

        # Empty list to store the detected keypoints
        points = []

        for i in range(nPoints):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]

            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # Scale the point to fit on the original image
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H

            if prob > threshold :
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

                # Add the point to the list if the probability is greater than the threshold
                points.append((int(x), int(y)))
            else :
                points.append(None)

        #to draw a binary image, first create a black image
        imgg = np.zeros((frameHeight, frameWidth ,3), np.uint8)

        # Draw Skeleton
        for pair in POSE_PAIRS:
            partA = pair[0]
            partB = pair[1]

            if points[partA] and points[partB]:
                cv2.line(imgg, points[partA], points[partB], (255, 255, 255), 5)
                #cv2.line(imgg, points[partA], points[partB], (255, 255, 255), 9)
                #cv2.circle(imgg, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

        gray = imgg.copy()
        gray = cv2.cvtColor(imgg, cv2.COLOR_RGB2GRAY)
        ret,thresh1 = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
        #image, contours,hierarchy = cv2.findContours(thresh1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        _,contours,hierarchy = cv2.findContours(thresh1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        x,y,w,h = cv2.boundingRect(cnt)
        crop = imgg[y:y+h,x:x+w]

        #Must create this folder first and rename it for each pose
        cv2.imwrite('fallOut/' + imageName, crop)
    except Exception as err:
        print(err)



In [26]:
def getAllImages(rootdir):
    """Gets all of the images in a folder and runs them through OpenPose.
    You just need to run this function with the folder of pose images"""
    path, dirs, files = next(os.walk(rootdir))
    file_count = len(files) - 1
    poseList = []

    i = 0
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            # print os.path.join(subdir, file)
            filepath = subdir + os.sep + file

            if filepath.endswith(".jpg"):
                i += 1
                #printProgressBar(i + 1, file_count, prefix='Progress:', suffix='Complete', length=50)
                print("processing image "+ str(i))
                convertImage(filepath,i)

#getAllImages('pose4a')

In [18]:
t = time.time()
print("Time at start: " + str((time.time() - t)))
getAllImages('standing')
print("Time taken: " + str((time.time() - t)))

Time at start: 6.699562072753906e-05
processing image 1
processing image 2
processing image 3
processing image 4
processing image 5
processing image 6
processing image 7
processing image 8
processing image 9
processing image 10
processing image 11
processing image 12
processing image 13
processing image 14
processing image 15
processing image 16
processing image 17
processing image 18
processing image 19
processing image 20
processing image 21
processing image 22
processing image 23
processing image 24
processing image 25
processing image 26
processing image 27
processing image 28
processing image 29
processing image 30
processing image 31
processing image 32
processing image 33
processing image 34
processing image 35
processing image 36
processing image 37
processing image 38
processing image 39
processing image 40
processing image 41
processing image 42
processing image 43
processing image 44
processing image 45
processing image 46
processing image 47
processing image 48
processing i

In [20]:
t = time.time()
print("Time at start: " + str((time.time() - t)))
getAllImages('walking')
print("Time taken: " + str((time.time() - t)))

Time at start: 0.00016188621520996094
processing image 1
processing image 2
processing image 3
processing image 4
processing image 5
processing image 6
processing image 7
processing image 8
processing image 9
processing image 10
processing image 11
processing image 12
processing image 13
processing image 14
processing image 15
processing image 16
processing image 17
processing image 18
processing image 19
processing image 20
processing image 21
processing image 22
processing image 23
processing image 24
processing image 25
processing image 26
processing image 27
processing image 28
processing image 29
processing image 30
processing image 31
processing image 32
processing image 33
processing image 34
processing image 35
processing image 36
processing image 37
processing image 38
processing image 39
processing image 40
processing image 41
processing image 42
processing image 43
processing image 44
processing image 45
processing image 46
processing image 47
processing image 48
processing 

In [27]:
t = time.time()
print("Time at start: " + str((time.time() - t)))
getAllImages('sitting')
print("Time taken: " + str((time.time() - t)))

Time at start: 6.4849853515625e-05
processing image 1
processing image 2
processing image 3
processing image 4
processing image 5
processing image 6
processing image 7
processing image 8
processing image 9
processing image 10
processing image 11
processing image 12
processing image 13
processing image 14
processing image 15
processing image 16
list index out of range
processing image 17
processing image 18
processing image 19
processing image 20
processing image 21
processing image 22
processing image 23
processing image 24
processing image 25
processing image 26
processing image 27
processing image 28
processing image 29
processing image 30
processing image 31
processing image 32
processing image 33
processing image 34
processing image 35
processing image 36
processing image 37
processing image 38
processing image 39
processing image 40
processing image 41
processing image 42
processing image 43
list index out of range
processing image 44
processing image 45
processing image 46
proces

In [29]:
t = time.time()
print("Time at start: " + str((time.time() - t)))
getAllImages('falling')
print("Time taken: " + str((time.time() - t)))

Time at start: 0.00016832351684570312
processing image 1
processing image 2
processing image 3
processing image 4
processing image 5
list index out of range
processing image 6
processing image 7
processing image 8
list index out of range
processing image 9
processing image 10
processing image 11
processing image 12
processing image 13
processing image 14
processing image 15
processing image 16
processing image 17
list index out of range
processing image 18
list index out of range
processing image 19
processing image 20
processing image 21
processing image 22
processing image 23
processing image 24
list index out of range
processing image 25
processing image 26
processing image 27
processing image 28
list index out of range
processing image 29
processing image 30
processing image 31
processing image 32
processing image 33
processing image 34
processing image 35
processing image 36
list index out of range
processing image 37
processing image 38
processing image 39
processing image 40
pro